In [206]:
! source ../venv/bin/activate

In [207]:
! which python

/Users/JMECKEL/development/projects/python/udacity_boarding_kiosk/venv/bin/python


In [208]:
import sys
print(sys.executable)

/Users/JMECKEL/development/projects/python/udacity_boarding_kiosk/venv/bin/python


In [209]:

import json
import pandas as pd
import os

In [210]:
manifest_file = "../starter/flight-manifest-csv/FlightManifest_withDoB.csv"

In [211]:
df = pd.read_csv(manifest_file)
df['DoBValidation'] = False
df['NameValidation'] = False
df['BoardingPassValidation'] = False
print(df)

   Passanger Name Carrier   Flight No.     Class           From       To  \
0    Sameer Kumar       UA         234   Economy  San Francisco  Chicago   
1  Radha S. Kumar       UA         234   Economy  San Francisco  Chicago   
2      James Webb       UA         234  Business  San Francisco  Chicago   
3    Libby Herold       UA         234  Business  San Francisco  Chicago   
4   James Jackson       UA         234   Economy  San Francisco  Chicago   
5  Avkash Chauhan       UA         234   Economy  San Francisco  Chicago   

             Date Baggage Seat Gate Boarding Time  Ticket No. Date of Birth  \
0  April 20, 2022     YES  34A   G1  10:00 AM PST    34236746    1990-01-25   
1  April 20, 2022     YES  34B   G1  10:00 AM PST    34236747    1970-12-15   
2  April 20, 2022      No   1A   G1  10:00 AM PST    34236748    1970-12-15   
3  April 20, 2022      No   3D   G1  10:00 AM PST    34236749    1996-02-10   
4  April 20, 2022     YES  25B   G1  10:00 AM PST    34236750    1956-10

In [212]:
for idx, row in df.iterrows():
    passanger_name = row['Passanger Name']
    last_name = passanger_name.split()[-1].strip().strip('.')  # Assuming last name is the last part of the passenger name
    first_name = ' '.join(passanger_name.split()[:-1]).strip().strip('.')
    print(f"First Name: {first_name}, Last Name: {last_name}")
    id_file_path = f"./results/id_{first_name}_{last_name}.json"
    if not os.path.exists(id_file_path):
        print(f"File {id_file_path} does not exist.")
        df.at[idx, 'DoBValidation'] = False
    else:
        with open(id_file_path, 'r') as f:
            data_from_id = json.load(f)
            if data_from_id['first_name'] != first_name:
                print(f"\nFirst name {data_from_id['first_name']} does not match expected {first_name}")
                df.at[idx, 'NameValidation'] = False
            else:
                if data_from_id['last_name'] != last_name:
                    print(f"Last name {data_from_id['last_name']} does not match expected {last_name}")
                    df.at[idx, 'NameValidation'] = False
                else:
                    df.at[idx, 'NameValidation'] = True
            if row.get("Date of Birth") == data_from_id['dob']:
                df.at[idx, 'DoBValidation'] = True
            else:
                print(f"Date of Birth {data_from_id['dob']} does not match expected {row['Date of Birth']}")
                df.at[idx, 'DoBValidation'] = False

    boarding_pass_file_path = f"./results/boarding_pass_{first_name}_{last_name}.json"
    if not os.path.exists(boarding_pass_file_path):
        print(f"File {boarding_pass_file_path} does not exist")
        df.at[idx, 'BoardingPassValidation'] = False
    else:
        with open(boarding_pass_file_path, 'r') as f:
            data_from_boarding_pass = json.load(f)
            if df.loc[idx, 'NameValidation'] and first_name == data_from_boarding_pass['first_name'] and last_name == data_from_boarding_pass['last_name']:
                df.at[idx, 'NameValidation'] = True
            else:
                print(f"Name on boarding pass {data_from_boarding_pass['first_name']} {data_from_boarding_pass['last_name']} does not match expected {first_name} {last_name}")
                df.at[idx, 'NameValidation'] = False
            if row.get("From") != data_from_boarding_pass['from']:
                df.at[idx, 'BoardingPassValidation'] = False
                print(f"From on boarding pass {data_from_boarding_pass['from']} does not match expected {row.get('From')}")
                print("------------------------")
                continue
            if row.get("To") != data_from_boarding_pass['to']:
                df.at[idx, 'BoardingPassValidation'] = False
                print(f"To on boarding pass {data_from_boarding_pass['to']} does not match expected {row.get('To')}")
                print("------------------------")
                continue
            if row.get("Gate") != data_from_boarding_pass['gate']:
                df.at[idx, 'BoardingPassValidation'] = False
                print(f"Gate on boarding pass {data_from_boarding_pass['gate']} does not match expected {row.get('Gate')}")
                print("------------------------")
                continue
            if str(row.get("Flight No.")).strip() != str(data_from_boarding_pass['fno']).strip():
                df.at[idx, 'BoardingPassValidation'] = False
                print(f"Flight No. on boarding pass {data_from_boarding_pass['fno']} does not match expected {row.get('Flight No.')}")
                continue
            if row.get("Seat") != data_from_boarding_pass['sno']:
                df.at[idx, 'BoardingPassValidation'] = False
                print(f"Seat on boarding pass {data_from_boarding_pass['sno']} does not match expected {row.get('Seat')}")
                print("------------------------")
                continue
            if row.get("Carrier ") != data_from_boarding_pass['carrier']:
                df.at[idx, 'BoardingPassValidation'] = False
                print(f"Carrier on boarding pass {data_from_boarding_pass['carrier']} does not match expected {row.get('Carrier ')}")
                print("------------------------")
                continue
            if row.get("Class")[0] != data_from_boarding_pass['class'][0]:
                df.at[idx, 'BoardingPassValidation'] = False
                print(f"Class on boarding pass {data_from_boarding_pass['class']} does not match expected {row.get('Class')}")
                print("------------------------")
                continue
            if row.get("Date") != data_from_boarding_pass['date']:
                df.at[idx, 'BoardingPassValidation'] = False
                print(f"Date on boarding pass {data_from_boarding_pass['date']} does not match expected {row.get('Date')}")
                print("------------------------")
                continue
            df.at[idx, 'BoardingPassValidation'] = True
            print(f"All Boarding-Pass validations passed for {first_name} {last_name}")
    print("------------------------")

First Name: Sameer, Last Name: Kumar
All Boarding-Pass validations passed for Sameer Kumar
------------------------
First Name: Radha S, Last Name: Kumar
File ./results/id_Radha S_Kumar.json does not exist.
Name on boarding pass Radha S Kumar does not match expected Radha S Kumar
All Boarding-Pass validations passed for Radha S Kumar
------------------------
First Name: James, Last Name: Webb
All Boarding-Pass validations passed for James Webb
------------------------
First Name: Libby, Last Name: Herold
All Boarding-Pass validations passed for Libby Herold
------------------------
First Name: James, Last Name: Jackson
All Boarding-Pass validations passed for James Jackson
------------------------
First Name: Avkash, Last Name: Chauhan
File ./results/id_Avkash_Chauhan.json does not exist.
Name on boarding pass Avkash Chauhan does not match expected Avkash Chauhan
All Boarding-Pass validations passed for Avkash Chauhan
------------------------


In [213]:
print(df)

   Passanger Name Carrier   Flight No.     Class           From       To  \
0    Sameer Kumar       UA         234   Economy  San Francisco  Chicago   
1  Radha S. Kumar       UA         234   Economy  San Francisco  Chicago   
2      James Webb       UA         234  Business  San Francisco  Chicago   
3    Libby Herold       UA         234  Business  San Francisco  Chicago   
4   James Jackson       UA         234   Economy  San Francisco  Chicago   
5  Avkash Chauhan       UA         234   Economy  San Francisco  Chicago   

             Date Baggage Seat Gate Boarding Time  Ticket No. Date of Birth  \
0  April 20, 2022     YES  34A   G1  10:00 AM PST    34236746    1990-01-25   
1  April 20, 2022     YES  34B   G1  10:00 AM PST    34236747    1970-12-15   
2  April 20, 2022      No   1A   G1  10:00 AM PST    34236748    1970-12-15   
3  April 20, 2022      No   3D   G1  10:00 AM PST    34236749    1996-02-10   
4  April 20, 2022     YES  25B   G1  10:00 AM PST    34236750    1956-10